In [1]:
import pandas as pd
import numpy as np

# 1. **데이터 설명**
1. pathway_train, pathway_test, pathway_valid : CCLE에서 pathway score 계산 한 데이터
2. ic_train, ic_test, ic_val : 민감군, 저항군 라벨
3. ccle : 유전자 raw data -> 민감군, 저항군 간 유전자 발현량이 가장 큰 유전자 3개를 genes에 저장
4. tcga: 환자 데이터로 약물 민간/저항 라벨 부착 예정

In [3]:
pathway_train = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/MK_1775/train_pathway_score_MK.1775.csv")
pathway_test = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/MK_1775/test_pathway_score_MK.1775.csv")
pathway_valid = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/MK_1775/val_pathway_score_MK.1775.csv")

ic_train = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/MK_1775/ic_train_MK.1775.csv")
ic_test = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/MK_1775/ic_test_MK.1775.csv")
ic_val = pd.read_csv("C:/Users/USER/비어플 의료/0415_final/pathway scores/MK_1775/ic_val_MK.1775.csv")

ccle = pd.read_csv("C:/Users/User/비어플 의료/#_filtered_CCLE_gene_expression.csv")
ccle.index = ccle["Unnamed: 0"]




In [4]:
pathway_train.set_index("SampleID", inplace= True)
pathway_test.set_index("SampleID", inplace= True)
pathway_valid.set_index("SampleID", inplace= True)

ic_train.set_index("Unnamed: 0", inplace=True)
ic_test.set_index("Unnamed: 0", inplace=True)
ic_val.set_index("Unnamed: 0", inplace=True)


In [5]:
genes = ["MAP4K1", "WAS", "CCND2"  ]

# 2. **전처리**
1. pathway score scale
2. pathway score에 대해 PCA 진행 후 90% 설명력까지 사용
3. train 기준으로 fjt하고 trest, valid, TCGA에는 transform만 해주기
4. ccle, tcga 유전자들도 sacle해주기
5. ic라벨 인코딩`

In [7]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# 스케일링 (z-score 정규화가 되어있더라도 PCA 전에 한 번 더 정규화 추천)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(pathway_train)
X_test_scaled = scaler.transform(pathway_test)
X_val_scaled = scaler.transform(pathway_valid)
# PCA 수행 (주성분 2개로 축소)
pca = PCA(n_components=0.90)
pathway_pca_train = pca.fit_transform(X_scaled)
pathway_pca_test  = pca.transform(X_test_scaled)
pathway_pca_valid = pca.transform(X_val_scaled)

In [8]:
pca_columns = [f"PC{i+1}" for i in range(pathway_pca_train.shape[1])]
pca_train_df_all = pd.DataFrame(pathway_pca_train, columns=pca_columns, index=pathway_train.index)
pca_train_df_all

pca_valid_df_all = pd.DataFrame(pathway_pca_valid, columns=pca_columns, index=pathway_valid.index)
pca_valid_df_all

pca_test_df_all = pd.DataFrame(pathway_pca_test, columns=pca_columns, index=pathway_test.index)
pca_test_df_all

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC36,PC37,PC38,PC39,PC40,PC41,PC42,PC43,PC44,PC45
SampleID,,,,,,,,,,,,,,,,,,,,,
ACH-000625,-0.961157,-2.729453,-2.194106,-1.251971,0.161081,-1.769774,-1.116364,-0.455696,1.145078,1.628460,...,0.282705,0.367340,1.414656,1.655861,-0.464536,1.662666,1.432288,0.605755,0.137033,0.810843
ACH-000035,10.202321,5.344649,1.044295,0.608017,1.239826,-1.679748,2.050368,0.083397,0.282861,-3.277724,...,-0.452738,0.091203,0.453718,0.177409,0.405277,-0.092370,-2.483702,-1.579641,-1.582506,0.780953
ACH-000223,5.716065,2.527377,2.409965,1.033711,-0.022761,-2.092238,0.592972,-0.789939,0.155504,-1.931718,...,0.695120,0.181542,0.148639,-0.374994,0.639405,0.515790,-1.178275,-1.494266,-0.938729,-0.183583
ACH-000978,2.203899,-5.945385,-6.112346,0.359050,-0.664158,-0.322713,-0.358260,-0.775414,-1.527140,-1.068789,...,-0.380958,0.481495,0.420651,0.374566,-1.035101,0.542309,0.845928,-0.973358,-0.499701,0.511595
ACH-000848,-1.911603,-7.792956,-2.816152,-0.911132,0.388582,0.198316,-1.184916,-0.687297,0.659374,1.498988,...,-1.762192,-1.407781,0.840383,0.443891,0.817133,0.319314,0.711264,-0.300761,1.606679,0.188222
ACH-000481,-4.868218,-8.355199,3.299067,-2.232769,-1.324490,0.004547,0.039877,-2.421998,1.198550,1.772096,...,-1.663617,-0.715203,-0.801773,-1.621507,-0.291178,0.259793,2.378134,-0.189426,0.526003,-0.352443
ACH-000858,-20.669690,-1.222050,2.068138,-1.972509,-1.366380,-1.348433,-0.306907,-0.797736,1.182978,1.742133,...,1.295734,1.881263,-1.764599,0.673925,-0.210999,-1.089846,0.767227,1.085237,1.738433,0.407294
ACH-000684,-1.343843,8.673382,1.754426,1.810389,1.865864,4.338824,-1.114318,1.193280,2.747319,-0.174246,...,-1.036438,-0.027490,-0.609358,-0.047012,0.328150,-1.150412,-0.791490,1.691721,-0.435155,0.943177
ACH-000943,-14.975195,-5.380253,-3.898431,-2.768222,-1.296940,2.420226,-4.975834,2.276823,0.932072,0.240016,...,-1.608300,0.959461,-1.424196,-0.387455,-1.003804,-0.231557,1.490219,1.319976,0.711265,-0.985795


In [9]:
ccle = ccle[genes]
ccle_expr_train = ccle.loc[pathway_train.index]
ccle_log_train = np.log2(ccle_expr_train+1)

scaler = StandardScaler()
ccle_scaled_train = pd.DataFrame(scaler.fit_transform(ccle_log_train),
                          columns = ccle_expr_train.columns,
                          index = ccle_expr_train.index)

ccle_expr_valid = ccle.loc[pathway_valid.index]
ccle_log_valid = np.log2(ccle_expr_valid+1)

ccle_scaled_valid = pd.DataFrame(scaler.transform(ccle_log_valid),
                          columns = ccle_expr_valid.columns,
                          index = ccle_expr_valid.index)

ccle_expr_test = ccle.loc[pathway_test.index]
ccle_log_test = np.log2(ccle_expr_test+1)
ccle_scaled_test = pd.DataFrame(scaler.transform(ccle_log_test),
                          columns = ccle_expr_test.columns,
                          index = ccle_expr_test.index)

In [10]:
X_train = pd.concat([pca_train_df_all, ccle_scaled_train], axis = 1)
X_valid = pd.concat([pca_valid_df_all, ccle_scaled_valid], axis = 1)
X_test = pd.concat([pca_test_df_all, ccle_scaled_test], axis = 1)

final_scaler = StandardScaler()
X_train_final = final_scaler.fit_transform(X_train)
X_valid_final = final_scaler.transform(X_valid)
X_test_final = final_scaler.transform(X_test)


In [11]:
# 숫자 라벨로 변환 (sensitive = 1, resistant = 0)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(ic_train.iloc[:,0])
y_valid = le.fit_transform(ic_val.iloc[:,0])
y_test = le.fit_transform(ic_test.iloc[:,0])

# **3. 모델링**
1. 로지스틱, 랜덤포레스트, SVM, KNN으로 진행
2. 전처리를 R로 해야했기에 pipeline으로 묶을 수 없어 cross validation이 힘듬
3. 평가지표는 AUC, F1 score 중점으로 사용
4. 수동으로 튜닝 후 성능이 제일 좋은 모델 선택 후 TCGA 라벨 예측

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

In [14]:
## Logistic Regression
lr = LogisticRegression(C=0.01, max_iter=10000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
y_proba_lr = lr.predict_proba(X_test)[:, 1]

## Random Forest
rf = RandomForestClassifier(n_estimators=200, max_depth=5, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:, 1]

## Support Vector Machine (SVM)
svm = SVC(probability=True, random_state=42)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
y_proba_svm = svm.predict_proba(X_test)[:, 1]

## K-Nearest Neighbors (KNN)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
y_proba_knn = knn.predict_proba(X_test)[:, 1]

# ── AdaBoost 추가 ──
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(
    n_estimators=50,
    random_state=42
)
ada.fit(X_train_final, y_train)

y_pred_ada   = ada.predict(X_test_final)
y_proba_ada  = ada.predict_proba(X_test_final)[:, 1]


C:\Users\USER\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [15]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

print("===== Validation Set Evaluation =====")

# Logistic Regression - Validation
y_pred_lr_valid = lr.predict(X_valid)
y_proba_lr_valid = lr.predict_proba(X_valid)[:, 1]
print("\n--- Logistic Regression ---")
print(classification_report(y_valid, y_pred_lr_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_lr_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_lr_valid))

# Random Forest - Validation
y_pred_rf_valid = rf.predict(X_valid)
y_proba_rf_valid = rf.predict_proba(X_valid)[:, 1]
print("\n--- Random Forest ---")
print(classification_report(y_valid, y_pred_rf_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_rf_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_rf_valid))

# Support Vector Machine - Validation
y_pred_svm_valid = svm.predict(X_valid)
y_proba_svm_valid = svm.predict_proba(X_valid)[:, 1]
print("\n--- Support Vector Machine ---")
print(classification_report(y_valid, y_pred_svm_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_svm_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_svm_valid))

# K-Nearest Neighbors - Validation
y_pred_knn_valid = knn.predict(X_valid)
y_proba_knn_valid = knn.predict_proba(X_valid)[:, 1]
print("\n--- K-Nearest Neighbors ---")
print(classification_report(y_valid, y_pred_knn_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_knn_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_knn_valid))
# AdaBoost - Validation
y_pred_ada_valid   = ada.predict(X_valid_final)
y_proba_ada_valid  = ada.predict_proba(X_valid_final)[:, 1]
print("\n--- AdaBoost ---")
print(classification_report(y_valid, y_pred_ada_valid, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_valid, y_proba_ada_valid))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred_ada_valid))


===== Validation Set Evaluation =====

--- Logistic Regression ---
              precision    recall  f1-score   support

   resistant       0.44      0.35      0.39        23
   sensitive       0.55      0.64      0.59        28

    accuracy                           0.51        51
   macro avg       0.49      0.50      0.49        51
weighted avg       0.50      0.51      0.50        51

ROC-AUC: 0.4751552795031056
Confusion matrix:
 [[ 8 15]
 [10 18]]

--- Random Forest ---
              precision    recall  f1-score   support

   resistant       0.50      0.22      0.30        23
   sensitive       0.56      0.82      0.67        28

    accuracy                           0.55        51
   macro avg       0.53      0.52      0.48        51
weighted avg       0.53      0.55      0.50        51

ROC-AUC: 0.5248447204968945
Confusion matrix:
 [[ 5 18]
 [ 5 23]]

--- Support Vector Machine ---
              precision    recall  f1-score   support

   resistant       0.36      0.35    

In [16]:
from sklearn.ensemble import VotingClassifier
# ── Soft Voting Ensemble ──
voting_clf = VotingClassifier(
    estimators=[
        ('lr', lr),
        ('rf', rf),
        ('svm', svm),
        ('knn', knn),
        ('ada', ada)
    ],
    voting='soft'
)
voting_clf.fit(X_train_final, y_train)

y_pred_voting  = voting_clf.predict(X_test_final)
y_proba_voting = voting_clf.predict_proba(X_test_final)[:, 1]

print("\n===== Soft Voting Ensemble =====")
print(classification_report(y_test, y_pred_voting, target_names=le.classes_))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_voting))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_voting))





===== Soft Voting Ensemble =====
              precision    recall  f1-score   support

   resistant       0.33      0.22      0.26        23
   sensitive       0.50      0.64      0.56        28

    accuracy                           0.45        51
   macro avg       0.42      0.43      0.41        51
weighted avg       0.42      0.45      0.43        51

ROC-AUC: 0.5
Confusion matrix:
 [[ 5 18]
 [10 18]]


C:\Users\USER\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
